In [1]:
import pandas as pd
import numpy as np

print("Imported pandas and numpy")

Imported pandas and numpy


In [2]:
data = pd.read_csv('/Users/ukapo/Churn/Customer_churn.csv')
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
print("Rows:", data.shape[0])
print("Columns:", data.shape[1])
print("\nColumn names:")
print(data.columns.tolist())
print("\nUnique Values:")
print(data.nunique())

Rows: 7043
Columns: 21

Column names:
['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']

Unique Values:
customerID          7043
gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6531
Churn                  2
dtype: int64


In [4]:
for cols in data:
    print(cols)
    print(data[data[cols] == ' '].index)
#Shows only TotalCharges has empty values

customerID
Int64Index([], dtype='int64')
gender
Int64Index([], dtype='int64')
SeniorCitizen
Int64Index([], dtype='int64')
Partner
Int64Index([], dtype='int64')
Dependents
Int64Index([], dtype='int64')
tenure
Int64Index([], dtype='int64')
PhoneService
Int64Index([], dtype='int64')
MultipleLines
Int64Index([], dtype='int64')
InternetService
Int64Index([], dtype='int64')
OnlineSecurity
Int64Index([], dtype='int64')
OnlineBackup
Int64Index([], dtype='int64')
DeviceProtection
Int64Index([], dtype='int64')
TechSupport
Int64Index([], dtype='int64')
StreamingTV
Int64Index([], dtype='int64')
StreamingMovies
Int64Index([], dtype='int64')
Contract
Int64Index([], dtype='int64')
PaperlessBilling
Int64Index([], dtype='int64')
PaymentMethod
Int64Index([], dtype='int64')
MonthlyCharges
Int64Index([], dtype='int64')
TotalCharges
Int64Index([488, 753, 936, 1082, 1340, 3331, 3826, 4380, 5218, 6670, 6754], dtype='int64')
Churn
Int64Index([], dtype='int64')


C:\Users\ukapo\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [5]:
#Replacing empty values with Null
data['TotalCharges'] = data['TotalCharges'].replace(' ',np.nan)

#Check it
data[data['TotalCharges'] == ' '].index

#Drop null values
data = data[data['TotalCharges'].notnull()]
print("Rows:", data.shape[0])

#Reset the index, drop deletes old index
data = data.reset_index(drop = True)

Rows: 7032


In [6]:
#Convert to float type
data['TotalCharges'] = data['TotalCharges'].astype(float)

#Replace No internet to No
replace_cols = ['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']
for i in replace_cols:
    data[i] = data[i].replace('No internet service','No')

#Replace 1/0 with Yes/No in senior citizens
data['SeniorCitizen'] = data['SeniorCitizen'].replace(1,'Yes')
data['SeniorCitizen'] = data['SeniorCitizen'].replace(0,'No')

#Tenure to categorical values
def tenure_cat(data):
    if data['tenure'] <= 12 :
        return 'Tenure_0-12'
    elif (data['tenure'] > 12) & (data['tenure'] <= 24):
        return 'Tenure_12-24'
    elif (data['tenure'] > 24) & (data['tenure'] <= 48):
        return 'Tenure_24-48'
    elif (data['tenure'] > 48) & (data['tenure'] <= 60):
        return 'Tenure_48-60'
    elif data['tenure'] > 60:
        return 'Tenure_gt_60'

data['tenure'] = data.apply(lambda data:tenure_cat(data), axis = 1)
print('done')


done


In [7]:
#Separate churn and non churn customers
churn = data[data['Churn'] == 'Yes']
non_churn = data[data['Churn'] == 'No']

#Separate numerical and categorical values
Id_col = ['customerID']
target_col = ['Churn']
 
#Gives columns with less than 6 unique values in a list, categorical columns
cat_cols   = data.nunique()[data.nunique() < 6].keys().tolist()

cat_cols = [x for x in cat_cols if x not in target_col]

#Numerical columns
num_cols = [x for x in data.columns if x not in cat_cols + Id_col + target_col]

#Columns with 2 values
bin_cols   = data.nunique()[data.nunique() == 2].keys().tolist()

#Columns with more than 2 values
multi_cols = [x for x in cat_cols if x not in bin_cols]

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

le = LabelEncoder()
#Binary columns
for i in bin_cols:
    data[i] = le.fit_transform(data[i])
    
#Multi columns
data = pd.get_dummies(data,columns = multi_cols )

#scaling numerical columns
std = StandardScaler()


In [9]:
#scaling numerical columns
std = StandardScaler()

scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns=num_cols)

data_og = data.copy()

#Replacing num cols with scaled values
data = data.drop(columns = num_cols, axis = 1)
data = data.merge(scaled, left_index=True, right_index=True, how = "left")




In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
from sklearn.metrics import roc_auc_score

#Splitting test and train data
train,test = train_test_split(data, test_size = .25, random_state = 111)

cols = [x for x in data.columns if x not in Id_col + target_col]
train_X = train[cols]
train_Y = train[target_col]
test_X  = test[cols]
test_Y  = test[target_col]

log_reg = LogisticRegression()
log_reg.fit(train_X, train_Y.values.ravel())

predictions = log_reg.predict(test_X)
probabilities = log_reg.predict_proba(test_X)

print("Classification Report\n")
print(classification_report(test_Y,predictions))

print("Accuracy Score: ", accuracy_score(test_Y, predictions))

print("Area Under Curve: ", roc_auc_score(test_Y,predictions))

Classification Report

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      1268
           1       0.69      0.50      0.58       490

    accuracy                           0.80      1758
   macro avg       0.76      0.71      0.72      1758
weighted avg       0.79      0.80      0.79      1758

Accuracy Score:  0.7974971558589306
Area Under Curve:  0.7074824567050795


C:\Users\ukapo\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
print("Confusion matrix\n")
conf_matrix = confusion_matrix(test_Y ,predictions)
print(" TN     FN")
print(conf_matrix[0][0]," ",conf_matrix[0][1])
print(" TP     FP")
print(conf_matrix[1][0],"  ",conf_matrix[1][1])

# Recall = True positive rate = TP/(TP + FN)
# 1 is best

# Specificity = True negative rate = TN/(TN + FP)
# 1 is best

# Precision = Positive predictive rate = TP/(TP + FP)
# 1 is best

# False Positive rate = FP/(TN + FP)
# 0  is best

Confusion matrix

 TN     FN
1155   113
 TP     FP
243    247
